# Module 11: Final Project - Deploy ML Model to Cloud

**Difficulty**: ⭐⭐⭐ Advanced

**Estimated Time**: 120-180 minutes

**Prerequisites**: 
- All previous modules (00-10)
- Cloud account (AWS, Azure, or GCP)
- Understanding of ML deployment concepts

## Learning Objectives

By completing this capstone project, you will:
1. Build an end-to-end ML pipeline from data to deployment
2. Deploy a production-ready model to your chosen cloud platform
3. Implement monitoring, logging, and alerting
4. Apply cost optimization techniques
5. Create API endpoints for model serving
6. Document your deployment for production use

## Project Overview

### The Challenge

You are a Machine Learning Engineer tasked with deploying a customer churn prediction model for a telecommunications company. The model must:

**Business Requirements**:
- Predict customer churn with >85% accuracy
- Provide predictions within 100ms
- Handle 1000+ daily predictions
- Cost <$100/month for infrastructure
- 99.9% uptime SLA

**Technical Requirements**:
- RESTful API for predictions
- Real-time monitoring dashboard
- Automated alerting for errors
- A/B testing capability
- Comprehensive documentation

### Deployment Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                   Cloud ML Deployment Pipeline                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Data                Training            Deployment             │
│  ┌────────┐         ┌────────┐         ┌──────────┐           │
│  │  S3 /  │────────▶│ Train  │────────▶│ Endpoint │           │
│  │ Blob / │         │  Job   │         │  (API)   │           │
│  │  GCS   │         └────────┘         └──────────┘           │
│  └────────┘              │                    │                 │
│                          ▼                    ▼                 │
│                   ┌────────────┐      ┌──────────────┐        │
│                   │   Model    │      │  Monitoring  │        │
│                   │  Registry  │      │  Dashboard   │        │
│                   └────────────┘      └──────────────┘        │
│                                              │                  │
│                                              ▼                  │
│                                       ┌──────────────┐         │
│                                       │    Alerts    │         │
│                                       │  (Email/SMS) │         │
│                                       └──────────────┘         │
└─────────────────────────────────────────────────────────────────┘
```

## Part 1: Setup and Data Preparation

### 1.1 Choose Your Cloud Platform

This project can be completed on AWS, Azure, or GCP. Choose based on:
- Available free credits
- Target job market
- Existing familiarity

**Platform-Specific Guides**:
- AWS: Use Modules 01-02
- Azure: Use Modules 03-04  
- GCP: Use Modules 05-06

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
import joblib
import json
import warnings
from datetime import datetime
from pathlib import Path

# Configure visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Libraries imported successfully")
print(f"Project initialized at: {datetime.now()}")

In [ ]:
# Configuration - Update with your cloud platform choice

PROJECT_CONFIG = {
    'platform': 'AWS',  # Options: 'AWS', 'Azure', 'GCP'
    'project_name': 'customer-churn-predictor',
    'model_version': 'v1.0',
    'deployment_environment': 'production',  # 'development' or 'production'
    
    # Performance targets
    'target_accuracy': 0.85,
    'max_latency_ms': 100,
    'daily_predictions': 1000,
    'monthly_budget': 100,  # USD
    
    # Paths
    'data_dir': 'data',
    'model_dir': 'models',
    'logs_dir': 'logs'
}

# Create necessary directories
for dir_path in [PROJECT_CONFIG['data_dir'], PROJECT_CONFIG['model_dir'], PROJECT_CONFIG['logs_dir']]:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print("Project Configuration:")
print(json.dumps(PROJECT_CONFIG, indent=2))

### 1.2 Load and Explore Data

We'll create a synthetic telecom customer dataset for this project.

In [ ]:
# Generate synthetic customer churn dataset

def generate_telecom_data(n_samples=10000):
    """
    Generate synthetic telecom customer data for churn prediction.
    
    Features:
    - Customer demographics
    - Account information
    - Usage patterns
    - Service details
    """
    np.random.seed(RANDOM_STATE)
    
    # Demographics
    tenure_months = np.random.randint(1, 72, n_samples)
    age = np.random.randint(18, 80, n_samples)
    is_senior = (age >= 65).astype(int)
    
    # Account info
    monthly_charges = np.random.uniform(20, 120, n_samples)
    total_charges = monthly_charges * tenure_months + np.random.normal(0, 50, n_samples)
    total_charges = np.maximum(total_charges, 0)  # No negative charges
    
    # Contract type: 0=Month-to-month, 1=One year, 2=Two year
    contract = np.random.choice([0, 1, 2], n_samples, p=[0.55, 0.25, 0.20])
    
    # Services
    internet_service = np.random.choice([0, 1, 2], n_samples, p=[0.2, 0.4, 0.4])  # 0=No, 1=DSL, 2=Fiber
    phone_service = np.random.choice([0, 1], n_samples, p=[0.1, 0.9])
    multiple_lines = np.random.choice([0, 1], n_samples, p=[0.5, 0.5])
    online_security = np.random.choice([0, 1], n_samples, p=[0.5, 0.5])
    tech_support = np.random.choice([0, 1], n_samples, p=[0.5, 0.5])
    
    # Payment
    paperless_billing = np.random.choice([0, 1], n_samples, p=[0.4, 0.6])
    payment_method = np.random.choice([0, 1, 2, 3], n_samples)  # Electronic check, Mailed check, Bank transfer, Credit card
    
    # Calculate churn probability based on features
    # Higher churn for: short tenure, month-to-month, high charges, no services
    churn_prob = (
        0.1  # Base rate
        + 0.3 * (tenure_months < 12)
        + 0.2 * (contract == 0)
        + 0.15 * (monthly_charges > 80)
        + 0.1 * (internet_service == 2)  # Fiber optic issues
        - 0.15 * (tech_support == 1)
        - 0.1 * (online_security == 1)
    )
    churn_prob = np.clip(churn_prob, 0, 1)
    churn = (np.random.random(n_samples) < churn_prob).astype(int)
    
    # Create DataFrame
    data = pd.DataFrame({
        'customer_id': [f'CUST{str(i).zfill(6)}' for i in range(n_samples)],
        'tenure_months': tenure_months,
        'age': age,
        'is_senior': is_senior,
        'monthly_charges': monthly_charges.round(2),
        'total_charges': total_charges.round(2),
        'contract_type': contract,
        'internet_service': internet_service,
        'phone_service': phone_service,
        'multiple_lines': multiple_lines,
        'online_security': online_security,
        'tech_support': tech_support,
        'paperless_billing': paperless_billing,
        'payment_method': payment_method,
        'churn': churn
    })
    
    return data

# Generate dataset
df = generate_telecom_data(10000)

# Save to disk
data_file = Path(PROJECT_CONFIG['data_dir']) / 'telecom_churn.csv'
df.to_csv(data_file, index=False)

print(f"Dataset generated: {len(df)} customers")
print(f"Saved to: {data_file}")
print(f"\nDataset shape: {df.shape}")
print(f"\nChurn rate: {df['churn'].mean()*100:.2f}%")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Exploratory Data Analysis

# Basic statistics
print("Dataset Summary:")
print("="*60)
print(df.describe())

# Check for missing values
print(f"\nMissing values:")
print(df.isnull().sum())

# Feature distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Feature Distributions', fontsize=16, fontweight='bold')

# Tenure
axes[0, 0].hist([df[df['churn']==0]['tenure_months'], 
                 df[df['churn']==1]['tenure_months']], 
                label=['No Churn', 'Churn'], bins=20, alpha=0.7)
axes[0, 0].set_xlabel('Tenure (months)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend()
axes[0, 0].set_title('Tenure Distribution')

# Monthly Charges
axes[0, 1].hist([df[df['churn']==0]['monthly_charges'], 
                 df[df['churn']==1]['monthly_charges']], 
                label=['No Churn', 'Churn'], bins=20, alpha=0.7)
axes[0, 1].set_xlabel('Monthly Charges ($)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend()
axes[0, 1].set_title('Monthly Charges Distribution')

# Contract Type
contract_churn = df.groupby('contract_type')['churn'].mean()
axes[0, 2].bar(contract_churn.index, contract_churn.values, alpha=0.7)
axes[0, 2].set_xlabel('Contract Type')
axes[0, 2].set_ylabel('Churn Rate')
axes[0, 2].set_title('Churn Rate by Contract Type')
axes[0, 2].set_xticks([0, 1, 2])
axes[0, 2].set_xticklabels(['Month-to-month', 'One year', 'Two year'])

# Internet Service
internet_churn = df.groupby('internet_service')['churn'].mean()
axes[1, 0].bar(internet_churn.index, internet_churn.values, alpha=0.7)
axes[1, 0].set_xlabel('Internet Service')
axes[1, 0].set_ylabel('Churn Rate')
axes[1, 0].set_title('Churn Rate by Internet Service')
axes[1, 0].set_xticks([0, 1, 2])
axes[1, 0].set_xticklabels(['No', 'DSL', 'Fiber'])

# Tech Support
tech_churn = df.groupby('tech_support')['churn'].mean()
axes[1, 1].bar(tech_churn.index, tech_churn.values, alpha=0.7)
axes[1, 1].set_xlabel('Tech Support')
axes[1, 1].set_ylabel('Churn Rate')
axes[1, 1].set_title('Churn Rate by Tech Support')
axes[1, 1].set_xticks([0, 1])
axes[1, 1].set_xticklabels(['No', 'Yes'])

# Overall churn rate
churn_counts = df['churn'].value_counts()
axes[1, 2].pie(churn_counts, labels=['No Churn', 'Churn'], autopct='%1.1f%%', startangle=90)
axes[1, 2].set_title('Overall Churn Rate')

plt.tight_layout()
plt.show()

print("\n✅ EDA complete")

## Part 2: Model Development

### 2.1 Data Preprocessing

In [ ]:
# Prepare features and target

# Drop customer_id (not a feature)
feature_columns = [col for col in df.columns if col not in ['customer_id', 'churn']]
X = df[feature_columns]
y = df['churn']

print(f"Features: {len(feature_columns)}")
print(f"Feature names: {feature_columns}")
print(f"\nTarget variable (churn): {y.value_counts()}")
print(f"Class balance: {y.value_counts(normalize=True)}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"\nTraining set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Feature scaling (important for some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Data preprocessing complete")

### 2.2 Model Training and Selection

We'll train multiple models and select the best one.

In [ ]:
# Train multiple models

models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE)
}

results = {}

print("Training models...\n")
print("="*80)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  AUC:       {auc:.4f}")
    
    # Check if meets target
    if accuracy >= PROJECT_CONFIG['target_accuracy']:
        print(f"  ✅ Meets accuracy target ({PROJECT_CONFIG['target_accuracy']})")
    else:
        print(f"  ❌ Below accuracy target ({PROJECT_CONFIG['target_accuracy']})")

print("\n" + "="*80)

In [ ]:
# Compare models visually

# Metrics comparison
metrics_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [r['accuracy'] for r in results.values()],
    'Precision': [r['precision'] for r in results.values()],
    'Recall': [r['recall'] for r in results.values()],
    'F1 Score': [r['f1'] for r in results.values()],
    'AUC': [r['auc'] for r in results.values()]
})

print("\nModel Comparison:")
print(metrics_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Metrics comparison
metrics_df.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1 Score']].plot(
    kind='bar', ax=axes[0], alpha=0.8
)
axes[0].set_title('Model Performance Comparison', fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_ylim(0, 1)
axes[0].legend(loc='lower right')
axes[0].grid(axis='y', alpha=0.3)
axes[0].axhline(y=PROJECT_CONFIG['target_accuracy'], color='red', 
                linestyle='--', label=f"Target ({PROJECT_CONFIG['target_accuracy']})")

# ROC curves
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
    axes[1].plot(fpr, tpr, label=f"{name} (AUC={result['auc']:.3f})")

axes[1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curves', fontweight='bold')
axes[1].legend(loc='lower right')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Select best model based on F1 score (balance of precision and recall)
best_model_name = max(results.items(), key=lambda x: x[1]['f1'])[0]
best_model = results[best_model_name]['model']

print(f"\n🏆 Best model selected: {best_model_name}")
print(f"   F1 Score: {results[best_model_name]['f1']:.4f}")
print(f"   Accuracy: {results[best_model_name]['accuracy']:.4f}")

In [ ]:
# Detailed evaluation of best model

best_result = results[best_model_name]

# Confusion matrix
cm = confusion_matrix(y_test, best_result['y_pred'])

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title(f'Confusion Matrix - {best_model_name}', fontweight='bold')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')
axes[0].set_xticklabels(['No Churn', 'Churn'])
axes[0].set_yticklabels(['No Churn', 'Churn'])

# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    axes[1].barh(importance_df['Feature'], importance_df['Importance'])
    axes[1].set_xlabel('Importance')
    axes[1].set_title('Feature Importance', fontweight='bold')
    axes[1].invert_yaxis()
else:
    axes[1].text(0.5, 0.5, 'Feature importance\nnot available\nfor this model',
                ha='center', va='center', fontsize=12)
    axes[1].axis('off')

plt.tight_layout()
plt.show()

# Classification report
print(f"\nClassification Report - {best_model_name}:")
print(classification_report(y_test, best_result['y_pred'], 
                           target_names=['No Churn', 'Churn']))

### 2.3 Save Model for Deployment

In [ ]:
# Save model and preprocessing artifacts

model_dir = Path(PROJECT_CONFIG['model_dir'])

# Save model
model_file = model_dir / f"churn_model_{PROJECT_CONFIG['model_version']}.joblib"
joblib.dump(best_model, model_file)
print(f"✅ Model saved: {model_file}")

# Save scaler
scaler_file = model_dir / f"scaler_{PROJECT_CONFIG['model_version']}.joblib"
joblib.dump(scaler, scaler_file)
print(f"✅ Scaler saved: {scaler_file}")

# Save feature names
feature_file = model_dir / f"features_{PROJECT_CONFIG['model_version']}.json"
with open(feature_file, 'w') as f:
    json.dump({
        'features': feature_columns,
        'model_type': best_model_name,
        'version': PROJECT_CONFIG['model_version']
    }, f, indent=2)
print(f"✅ Features saved: {feature_file}")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'model_version': PROJECT_CONFIG['model_version'],
    'training_date': datetime.now().isoformat(),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'features': feature_columns,
    'metrics': {
        'accuracy': float(best_result['accuracy']),
        'precision': float(best_result['precision']),
        'recall': float(best_result['recall']),
        'f1_score': float(best_result['f1']),
        'auc': float(best_result['auc'])
    },
    'target_accuracy': PROJECT_CONFIG['target_accuracy'],
    'meets_target': bool(best_result['accuracy'] >= PROJECT_CONFIG['target_accuracy'])
}

metadata_file = model_dir / f"model_metadata_{PROJECT_CONFIG['model_version']}.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved: {metadata_file}")

print(f"\n📦 Model artifacts ready for deployment")
print(f"   Total files: {len(list(model_dir.glob('*')))}")

## Part 3: Create Prediction Interface

### 3.1 Build Inference Pipeline

In [ ]:
# Create prediction pipeline

class ChurnPredictor:
    """
    Production-ready churn prediction pipeline.
    
    Handles:
    - Input validation
    - Preprocessing
    - Prediction
    - Error handling
    - Logging
    """
    
    def __init__(self, model_dir):
        """Load model artifacts"""
        self.model_dir = Path(model_dir)
        
        # Load metadata
        metadata_file = list(self.model_dir.glob('model_metadata_*.json'))[0]
        with open(metadata_file) as f:
            self.metadata = json.load(f)
        
        # Load model
        model_file = list(self.model_dir.glob('churn_model_*.joblib'))[0]
        self.model = joblib.load(model_file)
        
        # Load scaler (if needed)
        scaler_files = list(self.model_dir.glob('scaler_*.joblib'))
        self.scaler = joblib.load(scaler_files[0]) if scaler_files else None
        
        # Feature names
        self.feature_names = self.metadata['features']
        
        print(f"✅ ChurnPredictor initialized")
        print(f"   Model: {self.metadata['model_name']}")
        print(f"   Version: {self.metadata['model_version']}")
        print(f"   Accuracy: {self.metadata['metrics']['accuracy']:.4f}")
    
    def validate_input(self, data):
        """
        Validate input data.
        
        Args:
            data: dict or pd.DataFrame
        
        Returns:
            pd.DataFrame: Validated data
        """
        if isinstance(data, dict):
            data = pd.DataFrame([data])
        
        # Check required features
        missing_features = set(self.feature_names) - set(data.columns)
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")
        
        # Select only required features in correct order
        data = data[self.feature_names]
        
        return data
    
    def predict(self, data):
        """
        Make churn prediction.
        
        Args:
            data: Customer data (dict or DataFrame)
        
        Returns:
            dict: Prediction results
        """
        import time
        start_time = time.time()
        
        try:
            # Validate input
            df = self.validate_input(data)
            
            # Apply scaling if needed
            if self.scaler and self.metadata['model_name'] == 'Logistic Regression':
                X = self.scaler.transform(df)
            else:
                X = df.values
            
            # Predict
            prediction = int(self.model.predict(X)[0])
            probability = float(self.model.predict_proba(X)[0, 1])
            
            # Calculate latency
            latency_ms = (time.time() - start_time) * 1000
            
            return {
                'prediction': prediction,
                'churn_probability': probability,
                'latency_ms': latency_ms,
                'model_version': self.metadata['model_version'],
                'timestamp': datetime.now().isoformat()
            }
        
        except Exception as e:
            return {
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }

# Initialize predictor
predictor = ChurnPredictor(PROJECT_CONFIG['model_dir'])

In [ ]:
# Test prediction pipeline

# Sample customer (high churn risk)
high_risk_customer = {
    'tenure_months': 3,
    'age': 25,
    'is_senior': 0,
    'monthly_charges': 95.50,
    'total_charges': 285.00,
    'contract_type': 0,  # Month-to-month
    'internet_service': 2,  # Fiber
    'phone_service': 1,
    'multiple_lines': 0,
    'online_security': 0,
    'tech_support': 0,
    'paperless_billing': 1,
    'payment_method': 0
}

# Sample customer (low churn risk)
low_risk_customer = {
    'tenure_months': 48,
    'age': 55,
    'is_senior': 0,
    'monthly_charges': 45.00,
    'total_charges': 2160.00,
    'contract_type': 2,  # Two year
    'internet_service': 1,  # DSL
    'phone_service': 1,
    'multiple_lines': 1,
    'online_security': 1,
    'tech_support': 1,
    'paperless_billing': 0,
    'payment_method': 3
}

print("Testing prediction pipeline...\n")
print("="*60)

# Test high risk customer
result1 = predictor.predict(high_risk_customer)
print("\nHigh Risk Customer:")
print(json.dumps(result1, indent=2))
print(f"Risk Level: {'HIGH' if result1['churn_probability'] > 0.5 else 'LOW'}")

# Test low risk customer
result2 = predictor.predict(low_risk_customer)
print("\nLow Risk Customer:")
print(json.dumps(result2, indent=2))
print(f"Risk Level: {'HIGH' if result2['churn_probability'] > 0.5 else 'LOW'}")

print("\n" + "="*60)

# Check latency requirement
avg_latency = (result1['latency_ms'] + result2['latency_ms']) / 2
print(f"\nAverage latency: {avg_latency:.2f}ms")

if avg_latency < PROJECT_CONFIG['max_latency_ms']:
    print(f"✅ Meets latency requirement (<{PROJECT_CONFIG['max_latency_ms']}ms)")
else:
    print(f"❌ Exceeds latency requirement (<{PROJECT_CONFIG['max_latency_ms']}ms)")

## Part 4: Deployment Guide

### 4.1 Platform-Specific Deployment

Choose your deployment platform and follow the corresponding guide.

#### Option A: AWS SageMaker Deployment

**Steps**:

1. **Package Model for SageMaker**:
```bash
# Create model.tar.gz
tar -czf model.tar.gz -C models .
```

2. **Upload to S3**:
```python
import boto3
s3 = boto3.client('s3')
bucket = 'your-sagemaker-bucket'
s3.upload_file('model.tar.gz', bucket, 'models/churn/model.tar.gz')
```

3. **Create Inference Script** (`inference.py`):
```python
import json
import joblib
import numpy as np

def model_fn(model_dir):
    """Load model"""
    model = joblib.load(f"{model_dir}/churn_model_v1.0.joblib")
    return model

def input_fn(request_body, content_type):
    """Parse input"""
    if content_type == 'application/json':
        return json.loads(request_body)
    raise ValueError(f"Unsupported content type: {content_type}")

def predict_fn(input_data, model):
    """Make prediction"""
    # Your prediction logic here
    return model.predict([input_data])
```

4. **Deploy Endpoint**:
```python
from sagemaker.sklearn import SKLearnModel

model = SKLearnModel(
    model_data='s3://your-bucket/models/churn/model.tar.gz',
    role='your-sagemaker-role',
    entry_point='inference.py',
    framework_version='1.0-1'
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'
)
```

5. **Test Endpoint**:
```python
result = predictor.predict(test_data)
print(result)
```

**Cost Estimate**: ~$47/month for ml.t2.medium endpoint

#### Option B: Azure ML Deployment

**Steps**:

1. **Register Model**:
```python
from azureml.core import Workspace, Model

ws = Workspace.from_config()
model = Model.register(
    workspace=ws,
    model_path='models/churn_model_v1.0.joblib',
    model_name='churn-predictor'
)
```

2. **Create Scoring Script** (`score.py`):
```python
import json
import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('churn-predictor')
    model = joblib.load(model_path)

def run(data):
    data = json.loads(data)
    result = model.predict([data])
    return json.dumps({"prediction": int(result[0])})
```

3. **Deploy to Azure**:
```python
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(
    entry_script='score.py',
    environment=env
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1
)

service = Model.deploy(
    workspace=ws,
    name='churn-service',
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)
```

**Cost Estimate**: ~$30-40/month for basic ACI

#### Option C: GCP Vertex AI Deployment

**Steps**:

1. **Upload Model to GCS**:
```bash
gsutil cp models/* gs://your-bucket/churn-model/
```

2. **Create Custom Predictor**:
```python
from google.cloud import aiplatform

aiplatform.init(project='your-project', location='us-central1')

model = aiplatform.Model.upload(
    display_name='churn-predictor',
    artifact_uri='gs://your-bucket/churn-model/',
    serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest'
)
```

3. **Deploy Endpoint**:
```python
endpoint = model.deploy(
    deployed_model_display_name='churn-v1',
    machine_type='n1-standard-2',
    min_replica_count=1,
    max_replica_count=3
)
```

**Cost Estimate**: ~$50/month for n1-standard-2

## Part 5: Monitoring and Maintenance

### 5.1 Performance Monitoring

In [ ]:
# Create monitoring dashboard simulation

class ModelMonitor:
    """
    Monitor model performance in production.
    """
    
    def __init__(self):
        self.predictions = []
        self.latencies = []
        self.errors = []
    
    def log_prediction(self, result):
        """Log prediction metrics"""
        if 'error' in result:
            self.errors.append(result)
        else:
            self.predictions.append(result)
            self.latencies.append(result['latency_ms'])
    
    def get_metrics(self):
        """Calculate monitoring metrics"""
        if not self.predictions:
            return {'error': 'No predictions logged'}
        
        return {
            'total_predictions': len(self.predictions),
            'total_errors': len(self.errors),
            'error_rate': len(self.errors) / (len(self.predictions) + len(self.errors)),
            'avg_latency_ms': np.mean(self.latencies),
            'p50_latency_ms': np.percentile(self.latencies, 50),
            'p95_latency_ms': np.percentile(self.latencies, 95),
            'p99_latency_ms': np.percentile(self.latencies, 99),
            'max_latency_ms': np.max(self.latencies),
            'churn_rate': np.mean([p['prediction'] for p in self.predictions])
        }
    
    def check_alerts(self, thresholds):
        """Check if metrics exceed thresholds"""
        metrics = self.get_metrics()
        alerts = []
        
        if metrics['error_rate'] > thresholds.get('max_error_rate', 0.01):
            alerts.append(f"⚠️ High error rate: {metrics['error_rate']*100:.2f}%")
        
        if metrics['avg_latency_ms'] > thresholds.get('max_latency_ms', 100):
            alerts.append(f"⚠️ High latency: {metrics['avg_latency_ms']:.2f}ms")
        
        return alerts

# Simulate production traffic
monitor = ModelMonitor()

print("Simulating production traffic...\n")

# Simulate 100 predictions
for i in range(100):
    # Random customer
    customer = {
        'tenure_months': np.random.randint(1, 72),
        'age': np.random.randint(18, 80),
        'is_senior': np.random.choice([0, 1]),
        'monthly_charges': np.random.uniform(20, 120),
        'total_charges': np.random.uniform(100, 5000),
        'contract_type': np.random.choice([0, 1, 2]),
        'internet_service': np.random.choice([0, 1, 2]),
        'phone_service': np.random.choice([0, 1]),
        'multiple_lines': np.random.choice([0, 1]),
        'online_security': np.random.choice([0, 1]),
        'tech_support': np.random.choice([0, 1]),
        'paperless_billing': np.random.choice([0, 1]),
        'payment_method': np.random.choice([0, 1, 2, 3])
    }
    
    result = predictor.predict(customer)
    monitor.log_prediction(result)

# Display metrics
metrics = monitor.get_metrics()
print("Production Metrics:")
print("="*60)
print(f"Total Predictions: {metrics['total_predictions']}")
print(f"Total Errors: {metrics['total_errors']}")
print(f"Error Rate: {metrics['error_rate']*100:.2f}%")
print(f"\nLatency Statistics:")
print(f"  Average: {metrics['avg_latency_ms']:.2f}ms")
print(f"  P50: {metrics['p50_latency_ms']:.2f}ms")
print(f"  P95: {metrics['p95_latency_ms']:.2f}ms")
print(f"  P99: {metrics['p99_latency_ms']:.2f}ms")
print(f"  Max: {metrics['max_latency_ms']:.2f}ms")
print(f"\nPredicted Churn Rate: {metrics['churn_rate']*100:.2f}%")

# Check alerts
alerts = monitor.check_alerts({
    'max_error_rate': 0.01,
    'max_latency_ms': PROJECT_CONFIG['max_latency_ms']
})

print("\nAlerts:")
if alerts:
    for alert in alerts:
        print(f"  {alert}")
else:
    print("  ✅ All metrics within acceptable ranges")

## Part 6: Documentation and Handoff

### 6.1 Create Deployment Documentation

In [ ]:
# Generate deployment documentation

documentation = f"""
# Customer Churn Prediction Model - Deployment Documentation

## Model Overview

**Model Name**: {metadata['model_name']}
**Version**: {metadata['model_version']}
**Training Date**: {metadata['training_date']}
**Purpose**: Predict customer churn for proactive retention

## Performance Metrics

- **Accuracy**: {metadata['metrics']['accuracy']:.4f}
- **Precision**: {metadata['metrics']['precision']:.4f}
- **Recall**: {metadata['metrics']['recall']:.4f}
- **F1 Score**: {metadata['metrics']['f1_score']:.4f}
- **AUC**: {metadata['metrics']['auc']:.4f}

**Status**: {'✅ Meets' if metadata['meets_target'] else '❌ Below'} target accuracy ({PROJECT_CONFIG['target_accuracy']})

## Model Input

The model requires the following features:

```json
{{
  "tenure_months": "int (1-72)",
  "age": "int (18-80)",
  "is_senior": "int (0 or 1)",
  "monthly_charges": "float (20-120)",
  "total_charges": "float",
  "contract_type": "int (0=month-to-month, 1=one year, 2=two year)",
  "internet_service": "int (0=no, 1=DSL, 2=fiber)",
  "phone_service": "int (0 or 1)",
  "multiple_lines": "int (0 or 1)",
  "online_security": "int (0 or 1)",
  "tech_support": "int (0 or 1)",
  "paperless_billing": "int (0 or 1)",
  "payment_method": "int (0-3)"
}}
```

## Model Output

```json
{{
  "prediction": "int (0=no churn, 1=churn)",
  "churn_probability": "float (0.0-1.0)",
  "latency_ms": "float",
  "model_version": "string",
  "timestamp": "ISO 8601 datetime"
}}
```

## Deployment Configuration

**Platform**: {PROJECT_CONFIG['platform']}
**Environment**: {PROJECT_CONFIG['deployment_environment']}
**Monthly Budget**: ${PROJECT_CONFIG['monthly_budget']}
**SLA**: 99.9% uptime
**Max Latency**: {PROJECT_CONFIG['max_latency_ms']}ms

## Monitoring

Key metrics to monitor:
1. **Error Rate**: Should be <1%
2. **Latency**: P95 should be <{PROJECT_CONFIG['max_latency_ms']}ms
3. **Prediction Volume**: Expected {PROJECT_CONFIG['daily_predictions']}/day
4. **Predicted Churn Rate**: Monitor for drift

## Alerting

Set up alerts for:
- Error rate >1%
- P95 latency >{PROJECT_CONFIG['max_latency_ms']}ms
- Prediction volume drop >50%
- Churn rate drift >10% from baseline

## Maintenance

- **Retraining**: Monthly or when performance degrades
- **Data Updates**: Weekly data refresh recommended
- **Model Validation**: Compare predictions vs. actual outcomes

## Cost Optimization

Estimated monthly costs:
- Compute: $40-50 (auto-scaling)
- Storage: $5-10
- Data transfer: $5
- **Total**: ~${PROJECT_CONFIG['monthly_budget']}/month

## Support

For issues or questions:
- Technical: ml-team@company.com
- Business: retention-team@company.com

## Change Log

- **v1.0** ({metadata['training_date'][:10]}): Initial deployment

"""

# Save documentation
doc_file = Path(PROJECT_CONFIG['model_dir']) / 'DEPLOYMENT_GUIDE.md'
with open(doc_file, 'w') as f:
    f.write(documentation)

print("✅ Deployment documentation created")
print(f"   File: {doc_file}")
print("\nDocumentation Preview:")
print("="*60)
print(documentation[:1000] + "\n...")

## Project Summary and Next Steps

### What You've Accomplished

✅ **Built an end-to-end ML pipeline**:
- Generated and analyzed telecom customer data
- Trained and compared multiple models
- Selected best model based on business metrics

✅ **Created production-ready inference pipeline**:
- Input validation
- Error handling
- Performance monitoring
- Latency tracking

✅ **Prepared for cloud deployment**:
- Platform-specific deployment guides (AWS/Azure/GCP)
- Cost optimization strategies
- Monitoring and alerting setup

✅ **Documented everything**:
- Model metadata and artifacts
- Deployment guide
- API specifications
- Maintenance procedures

### Production Deployment Checklist

Before going live, ensure:

- [ ] Model meets accuracy target (>85%)
- [ ] Latency is acceptable (<100ms)
- [ ] Error handling is robust
- [ ] Monitoring dashboards are configured
- [ ] Alerts are set up
- [ ] Cost tracking is enabled
- [ ] Documentation is complete
- [ ] Team is trained on maintenance procedures
- [ ] Rollback plan is documented
- [ ] Security review completed

### Next Steps

1. **Deploy to Staging**:
   - Use smaller instance type
   - Test with synthetic traffic
   - Validate monitoring

2. **Production Deployment**:
   - Follow platform-specific guide
   - Start with canary deployment (10% traffic)
   - Monitor closely for 24-48 hours
   - Gradually increase to 100%

3. **Continuous Improvement**:
   - Collect feedback labels
   - Retrain monthly
   - A/B test new models
   - Optimize features

### Congratulations!

You've completed the Cloud ML Deployment learning path. You now have:
- Understanding of cloud ML platforms (AWS, Azure, GCP)
- Hands-on experience with deployment patterns
- Production-ready ML deployment skills
- Cost optimization knowledge
- Monitoring and maintenance expertise

**You're ready to deploy ML models to production!** 🚀